In [1]:
import torch
from torch.utils.data import Dataset
import pandas as pd
import cv2
from tqdm import tqdm
import numpy as np

class CASMECombinedDataset(Dataset):
    def __init__(self,path = '.',
                 img_sz = 128,
                 calculate_strain = False,
                 initialized_df = None):
        
        if initialized_df is None:
            print('Initializing CASME Combined Dataset...')
            self.df = pd.read_csv(path + '/' + 'combined_all_classes.csv')
            self.df['OpticalFlow'] = None
            self.path = path
            self.img_sz = img_sz
            for idx,row in tqdm(self.df.iterrows(),ascii = '='):
                prefix = self.__get_prefix(row)
                onset = self.__read_img(
                    prefix + str(row['Onset']) + '.jpg'
                )
                apex = self.__read_img(
                    prefix + str(row['Apex']) + '.jpg'
                )
                self.df.at[idx,'OpticalFlow'] = self.__calc_optical_flow(onset,apex)
                if calculate_strain:
                    self.df.at[idx,'OpticalFlow'] = self.__append_optical_strain(self.df.at[idx,'OpticalFlow'])
                self.df.at[idx,'Class'] = {'anger':0,'disgust':1,'fear':2,'happiness':3,'repression':4,'sadness':5,'surprise':6}[row['Class']]  
        else:
            self.df = initialized_df
    
    def __get_prefix(self,row):
        sub_sample = row['Subject'] + '/' + row['Sample'] + '/'
        if row['Dataset'] == 'casme1':
            return self.path + '/casme1_cropped/' + sub_sample + 'reg_' + row['Sample'] + '-'
        elif row['Dataset'] == 'casme2':
            return self.path + '/casme2_cropped/' + sub_sample + 'reg_img'
        elif row['Dataset'] == 'casme^2':
            return self.path + '/casme^2_cropped/' + sub_sample + 'img'
            
    def __read_img(self,name):
        return cv2.cvtColor(
            cv2.resize(
                cv2.imread(name,cv2.IMREAD_COLOR),
                (self.img_sz,self.img_sz),
                interpolation = cv2.INTER_CUBIC
            ),
            cv2.COLOR_BGR2GRAY
        )
    
    def __calc_optical_flow(self,onset,apex):
        return np.array(
            cv2.optflow.DualTVL1OpticalFlow_create().calc(onset,apex,None)
        ).transpose((2,0,1))
    
    def __append_optical_strain(self,flow):
        ux = cv2.Sobel(flow[0],cv2.CV_32F,1,0)
        uy = cv2.Sobel(flow[0],cv2.CV_32F,0,1)
        vx = cv2.Sobel(flow[1],cv2.CV_32F,1,0)
        vy = cv2.Sobel(flow[1],cv2.CV_32F,0,1)
        strain = np.sqrt(ux * ux + uy * uy + 0.5 * (vx + uy) * (vx + uy))
        return np.concatenate((flow,strain.reshape(1,self.img_sz,self.img_sz)),axis = 0)
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self,idx):
        return self.df.at[idx,'OpticalFlow'],int(self.df.at[idx,'Class'])
        
class LOSOGenerator():
    def __init__(self,dataset):
        self.data = dataset
        self.subjects = self.data.df[['Dataset','Subject']].drop_duplicates().reset_index()
        self.idx = 0
    
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.idx == len(self.subjects):
            raise StopIteration
        ds,sub = self.subjects.at[self.idx,'Dataset'],self.subjects.at[self.idx,'Subject']
        self.idx += 1
        train_df = self.data.df[(self.data.df.Dataset != ds) | (self.data.df.Subject != sub)] \
            .reset_index(drop=True)
        test_df = self.data.df[(self.data.df.Dataset == ds) & (self.data.df.Subject == sub)] \
            .reset_index(drop=True)
        return CASMECombinedDataset(initialized_df = train_df),CASMECombinedDataset(initialized_df = test_df)

In [2]:
from torch import nn
import torch

class SingleStream(nn.Module):
    def __init__(self,out_c):
        super(SingleStream,self).__init__()
        self.conv = nn.Conv2d(in_channels = 3,out_channels = out_c,kernel_size = 3,padding = 1)
        self.pool = nn.MaxPool2d(kernel_size = 3,stride = 3,padding = 1)
    
    def forward(self,x):
        x = self.conv(x)
        x = self.pool(x)
        return x

class STSTNet2(nn.Module):
    def __init__(self):
        super(STSTNet2,self).__init__()
        self.stream1 = SingleStream(3)
        self.stream2 = SingleStream(5)
        self.stream3 = SingleStream(8)
        self.pool = nn.MaxPool2d(kernel_size = 2,stride = 2)
        self.dense = nn.Sequential(
            nn.Linear(400,400),
            nn.Dropout(0,8),
            nn.ReLU(),
            nn.Linear(400,7)
        )

    def forward(self,x):
        x1 = self.stream1(x)
        x2 = self.stream2(x)
        x3 = self.stream3(x)
        x = torch.cat([x1,x2,x3],dim = 1)
        x = self.pool(x)
        x = x.view(-1,400)
        x = self.dense(x)
        return x

In [3]:
ststf = torch.load('maml_ststnet.pth',map_location = torch.device('cpu'))

/home/qwq/anaconda3/lib/python3.8/site-packages/torch/serialization.py:658: SourceChangeWarning: source code of class 'torch.nn.modules.container.ParameterList' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/qwq/anaconda3/lib/python3.8/site-packages/torch/nn/modules/container.py:434: UserWarning: Setting attributes on ParameterList is not supported.
  warnings.warn("Setting attributes on ParameterList is not supported.")


In [4]:
model = STSTNet2()

In [5]:
ststf

STSTNet(
  (vars): ParameterList(
      (0): Parameter containing: [torch.FloatTensor of size 3x3x3x3]
      (1): Parameter containing: [torch.FloatTensor of size 3]
      (2): Parameter containing: [torch.FloatTensor of size 5x3x3x3]
      (3): Parameter containing: [torch.FloatTensor of size 5]
      (4): Parameter containing: [torch.FloatTensor of size 8x3x3x3]
      (5): Parameter containing: [torch.FloatTensor of size 8]
      (6): Parameter containing: [torch.FloatTensor of size 400x400]
      (7): Parameter containing: [torch.FloatTensor of size 400]
      (8): Parameter containing: [torch.FloatTensor of size 5x400]
      (9): Parameter containing: [torch.FloatTensor of size 5]
  )
)

In [6]:
conv1 = model.stream1.conv.state_dict()
conv1['weight'] = ststf.vars[0]
conv1['bias'] = ststf.vars[1]
model.stream1.conv.load_state_dict(conv1)
conv2 = model.stream2.conv.state_dict()
conv2['weight'] = ststf.vars[2]
conv2['bias'] = ststf.vars[3]
model.stream2.conv.load_state_dict(conv2)
conv3 = model.stream3.conv.state_dict()
conv3['weight'] = ststf.vars[4]
conv3['bias'] = ststf.vars[5]
model.stream3.conv.load_state_dict(conv3)
fc1 = model.dense[0].state_dict()
conv3['weight'] = ststf.vars[6]
conv3['bias'] = ststf.vars[7]
model.dense[0].load_state_dict(fc1)

<All keys matched successfully>

In [7]:
import torch.nn.functional as F

class focal_loss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2, num_classes = 7, size_average=True):
        super(focal_loss,self).__init__()
        self.size_average = size_average
        if isinstance(alpha,list):
            assert len(alpha)==num_classes   
            self.alpha = torch.Tensor(alpha)
        else:
            assert alpha<1  
            self.alpha = torch.zeros(num_classes)
            self.alpha[0] += alpha
            self.alpha[1:] += (1-alpha)

        self.gamma = gamma

    def forward(self, preds, labels):
        # assert preds.dim()==2 and labels.dim()==1
        preds = preds.view(-1,preds.size(-1))
        self.alpha = self.alpha.to(preds.device)
        preds_softmax = F.softmax(preds, dim=1) 
        preds_logsoft = torch.log(preds_softmax)
        
        #focal_loss func, Loss = -α(1-yi)**γ *ce_loss(xi,yi)
        preds_softmax = preds_softmax.gather(1,labels.view(-1,1)) 
        preds_logsoft = preds_logsoft.gather(1,labels.view(-1,1))
        self.alpha = self.alpha.gather(0,labels.view(-1))
        # torch.pow((1-preds_softmax), self.gamma) 为focal loss中 (1-pt)**γ
        loss = -torch.mul(torch.pow((1-preds_softmax), self.gamma), preds_logsoft) 

        loss = torch.mul(self.alpha, loss.t())
        if self.size_average:
            loss = loss.mean()
        else:
            loss = loss.sum()
        return loss


In [8]:
import torch
from tqdm.notebook import tqdm
import numpy as np
from torch.utils.data import DataLoader
from torch import nn,optim
import pickle as pkl
from sklearn.metrics import recall_score,f1_score
from copy import deepcopy

from_file = 1
dataset = None
data_filename = 'dataset28x28_allclass_strain.pkl'
print_debug_info = 1
n_epochs = 200
device = torch.device('cpu')
observed_epochs = set([i for i in range(0,n_epochs,20)])

if from_file:
    with open(data_filename,'rb') as f:
        dataset = pkl.load(f) 
else:    
    dataset = CASMECombinedDataset(path = '/home/qwq/MAML-MER/dataset',img_sz = 28,calculate_strain = 1)
    with open(data_filename,'wb') as f:
        pkl.dump(dataset,f)

def train(trainset,testset):
    loader = DataLoader(trainset,batch_size = len(trainset),shuffle = True)
    model0 = deepcopy(model)
    model0.train()
    optimizer = optim.Adam(model0.parameters(),lr = 1e-3)
    criterion = nn.CrossEntropyLoss(weight = torch.Tensor([10,1,10,2,2,10,2])).to(device)
    res_dict = {}
    for epoch in tqdm(range(n_epochs)):
        losses = []
        for idx,(x,y) in enumerate(loader):
            x,y = x.to(device),y.to(device)
            out = model0(x)
            loss = criterion(out,y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            losses.append(loss.cpu().detach().numpy())
        if epoch in observed_epochs:
            res = test(model0,testset)
            res_dict[epoch] = res
            model0.train()
            if print_debug_info:
                print('Epoch ' + str(epoch))
                print(np.mean(np.array(losses)))
                print(res)
            
    return res_dict
    
def test(model0,testset):
    loader = DataLoader(testset,batch_size = 1,shuffle = True)
    model0.eval()
    y,pred = [],[]
    for idx,(x0,y0) in enumerate(loader):
        x0,y0 = torch.Tensor(x0).to(device),y0.numpy()
        y.append(y0[0])
        pred0 = model0(x0).cpu().detach().numpy()
        pred.append(np.where(pred0[0] == np.max(pred0[0]))[0][0])
    y,pred = np.array(y),np.array(pred)
    print(y,pred)
    return get_metric(y,pred)

def get_metric(y,pred):
    uar = recall_score(y,pred,average = 'macro')
    uf1 = f1_score(y,pred,average = 'macro')
    return uar,uf1

def loso():
    print('Start cross validation.')
    results = []
    for idx,(trainset,testset) in tqdm(enumerate(LOSOGenerator(dataset))):
        print('Test ' + str(idx))
        res = train(trainset,testset)
        results.append(res)
        e1,(max_uar,_) = sorted(list(res.items()),reverse = True,key = lambda x:x[1][0])[0]
        e2,(_,max_uf1) = sorted(list(res.items()),reverse = True,key = lambda x:x[1][1])[0]
        print('Max UAR:' + str(max_uar) + ' at epoch ' + str(e1))
        print('Max UF1:' + str(max_uf1) + ' at epoch ' + str(e2))
    
    print('Cross validation finished.')
    mean_uar = lambda epoch:np.mean(np.array([dic[epoch][0] for dic in results]))
    mean_uf1 = lambda epoch:np.mean(np.array([dic[epoch][1] for dic in results]))
    e1 = sorted(list(observed_epochs),reverse = True,key = mean_uar)[0]
    max_mean_uar = mean_uar(e1)
    e2 = sorted(list(observed_epochs),reverse = True,key = mean_uar)[0]
    max_mean_uf1 = mean_uf1(e2)
    print('Max mean UAR:' + str(max_mean_uar) + ' at epoch ' + str(e1))
    print('Max mean UF1:' + str(max_mean_uf1) + ' at epoch ' + str(e2))
    
    with open(train_process_filename,'wb') as f:
        pkl.dump(results,f)
    
if __name__=='__main__':
    loso()

Start cross validation.


Test 0


/home/qwq/anaconda3/lib/python3.8/site-packages/torch/autograd/__init__.py:130: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  Variable._execution_engine.run_backward(
/home/qwq/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[1 0 3] [4 4 4]
Epoch 0
1.9523461
(0.0, 0.0)
[0 3 1] [4 1 5]
Epoch 20
1.0763148
(0.0, 0.0)
[1 0 3] [1 1 3]
Epoch 40
0.6217654
(0.6666666666666666, 0.5555555555555555)
[0 3 1] [1 3 3]
Epoch 60
0.3608204
(0.3333333333333333, 0.2222222222222222)
[3 0 1] [3 1 3]
Epoch 80
0.20806886
(0.3333333333333333, 0.2222222222222222)
[1 3 0] [1 3 1]
Epoch 100
0.121324636
(0.6666666666666666, 0.5555555555555555)
[1 0 3] [1 1 3]
Epoch 120
0.07338748
(0.6666666666666666, 0.5555555555555555)
[3 0 1] [1 1 1]
Epoch 140
0.04544264
(0.3333333333333333, 0.16666666666666666)
[3 1 0] [1 1 1]
Epoch 160
0.029074276
(0.3333333333333333, 0.16666666666666666)
[1 0 3] [1 1 1]
Epoch 180
0.019534722
(0.3333333333333333, 0.16666666666666666)

Max UAR:0.6666666666666666 at epoch 40
Max UF1:0.5555555555555555 at epoch 40
Test 1


[3 1 1 3 3 3 3 6] [4 4 4 4 4 6 1 4]
Epoch 0
1.953066
(0.0, 0.0)
[3 3 3 1 6 3 3 1] [4 1 0 1 5 0 4 6]
Epoch 20
1.0693387
(0.08333333333333333, 0.08333333333333333)
[3 3 6 3 1 3 1 3] [0 6 2 1 6 3 1 1]
Epoch 40
0.6097471
(0.13999999999999999, 0.14666666666666667)
[3 3 3 1 6 3 3 1] [6 1 1 1 2 6 3 6]
Epoch 60
0.3511306
(0.175, 0.18333333333333335)
[3 6 3 3 3 1 3 1] [3 2 6 1 6 6 1 1]
Epoch 80
0.20397268
(0.175, 0.18333333333333335)
[3 3 3 3 3 1 6 1] [6 1 6 1 3 6 2 1]
Epoch 100
0.120171286
(0.175, 0.18333333333333335)
[3 3 3 1 3 1 6 3] [1 3 6 1 6 6 2 1]
Epoch 120
0.073312216
(0.175, 0.18333333333333335)
[3 3 6 3 3 1 3 1] [6 6 2 1 1 1 3 6]
Epoch 140
0.04543516
(0.175, 0.18333333333333335)
[1 3 3 3 1 3 3 6] [1 3 6 1 6 4 6 2]
Epoch 160
0.028924566
(0.13999999999999999, 0.16666666666666669)
[1 3 3 3 6 1 3 3] [1 6 6 1 2 6 4 3]
Epoch 180
0.01933351
(0.13999999999999999, 0.16666666666666669)

Max UAR:0.175 at epoch 60
Max UF1:0.18333333333333335 at epoch 60
Test 2


...

[4 1 1 4 3 1 1 4] [1 4 6 4 4 4 4 6]
Epoch 0
1.953633
(0.08333333333333333, 0.0625)
[4 1 1 1 4 4 1 3] [1 1 1 1 1 3 1 4]
Epoch 20
1.0694902
(0.3333333333333333, 0.26666666666666666)
[3 4 4 1 4 1 1 1] [1 3 1 1 1 1 1 1]
Epoch 40
0.6121058
(0.3333333333333333, 0.24242424242424243)
[1 4 4 1 1 4 3 1] [1 1 3 1 1 1 4 1]
Epoch 60
0.35582688
(0.3333333333333333, 0.26666666666666666)
[3 4 1 4 1 1 1 4] [4 3 1 4 1 1 1 1]
Epoch 80
0.20594524
(0.4444444444444444, 0.42962962962962964)
[1 1 1 1 4 4 4 3] [1 1 1 1 3 1 4 4]
Epoch 100
0.120399676
(0.4444444444444444, 0.42962962962962964)
[4 1 3 4 1 1 4 1] [3 1 4 4 1 1 1 1]
Epoch 120
0.0727131
(0.4444444444444444, 0.42962962962962964)
[1 4 1 1 3 4 1 4] [1 4 1 1 4 3 1 1]
Epoch 140
0.044850502
(0.4444444444444444, 0.42962962962962964)
[1 1 3 1 4 4 1 4] [1 1 4 1 4 3 1 1]
Epoch 160
0.02854599
(0.4444444444444444, 0.42962962962962964)
[4 4 1 1 1 3 1 4] [1 3 1 1 1 4 1 4]
Epoch 180
0.019113706
(0.4444444444444444, 0.42962962962962964)

Max UAR:0.4444444444444444 at

[1 6 6 1 6 6] [4 4 1 4 4 4]
Epoch 0
1.9536514
(0.0, 0.0)
[6 1 6 6 6 1] [6 4 4 1 4 3]
Epoch 20
1.0650655
(0.0625, 0.1)
[1 6 6 6 1 6] [3 6 1 6 4 4]
Epoch 40
0.6010424
(0.125, 0.16666666666666666)
[6 1 1 6 6 6] [1 1 3 6 6 4]
Epoch 60
0.34386048
(0.25, 0.29166666666666663)
[6 6 6 1 1 6] [1 6 3 3 1 4]
Epoch 80
0.19954842
(0.1875, 0.225)
[6 6 6 6 1 1] [6 3 4 1 3 1]
Epoch 100
0.11725458
(0.1875, 0.225)
[6 6 1 1 6 6] [4 6 3 1 1 3]
Epoch 120
0.07060083
(0.1875, 0.225)
[6 6 6 6 1 1] [1 6 4 3 1 3]
Epoch 140
0.043586813
(0.1875, 0.225)
[6 1 1 6 6 6] [6 1 3 1 4 3]
Epoch 160
0.027908126
(0.1875, 0.225)
[6 6 1 6 6 1] [4 1 1 6 3 3]
Epoch 180
0.01884133
(0.1875, 0.225)

Max UAR:0.25 at epoch 60
Max UF1:0.29166666666666663 at epoch 60
Test 52


[6 1 1 6 1] [4 4 4 4 4]
Epoch 0
1.9525872
(0.0, 0.0)
[1 6 1 1 6] [6 6 5 1 6]
Epoch 20
1.0677569
(0.4444444444444444, 0.43333333333333335)
[1 6 6 1 1] [6 6 6 6 1]
Epoch 40
0.610593
(0.6666666666666666, 0.5833333333333333)
[1 6 6 1 1] [1 6 6 6 6]
Epoch 60
0.35213664
(0.6666666666666666, 0.5833333333333333)
[1 1 1 6 6] [1 6 1 6 6]
Epoch 80
0.20291641
(0.8333333333333333, 0.8)
[1 6 6 1 1] [1 6 6 6 3]
Epoch 100
0.11872127
(0.4444444444444444, 0.43333333333333335)
[1 6 1 6 1] [3 6 1 6 6]
Epoch 120
0.0721554
(0.4444444444444444, 0.43333333333333335)
[1 6 1 1 6] [1 6 6 3 6]
Epoch 140
0.044680562
(0.4444444444444444, 0.43333333333333335)
[1 1 6 6 1] [3 1 6 6 6]
Epoch 160
0.028505962
(0.4444444444444444, 0.43333333333333335)
[1 6 1 6 1] [3 6 6 6 1]
Epoch 180
0.019098286
(0.4444444444444444, 0.43333333333333335)

Max UAR:0.8333333333333333 at epoch 80
Max UF1:0.8 at epoch 80
Test 53


[1 5 3 1 1 1 1 1 3 1 1 1] [4 6 4 4 4 6 4 4 4 4 6 4]
Epoch 0
1.9542766
(0.0, 0.0)
[1 1 5 3 1 1 1 1 1 1 1 3] [0 1 6 5 1 1 4 1 1 6 1 1]
Epoch 20
1.0732961
(0.1111111111111111, 0.125)
[1 1 1 5 3 1 3 1 1 1 1 1] [1 1 1 6 1 3 5 1 1 1 1 4]
Epoch 40
0.615895
(0.15555555555555556, 0.1647058823529412)
[1 1 5 1 1 1 1 1 3 1 1 3] [1 1 2 1 1 1 1 3 1 1 1 3]
Epoch 60
0.35692102
(0.3472222222222222, 0.3472222222222222)
[1 3 5 1 3 1 1 1 1 1 1 1] [1 3 2 3 1 1 1 1 1 1 1 1]
Epoch 80
0.2084268
(0.3472222222222222, 0.3472222222222222)
[1 3 1 1 1 1 1 3 1 5 1 1] [1 3 3 1 1 1 1 1 1 2 3 1]
Epoch 100
0.12320489
(0.3194444444444444, 0.3058823529411765)
[1 1 3 3 1 1 1 5 1 1 1 1] [3 1 3 1 1 3 1 2 1 1 1 1]
Epoch 120
0.07486686
(0.3194444444444444, 0.3058823529411765)
[3 1 1 1 1 1 1 5 3 1 1 1] [1 1 3 1 1 3 1 2 3 1 1 1]
Epoch 140
0.046431363
(0.3194444444444444, 0.3058823529411765)
[5 3 1 1 1 1 3 1 1 1 1 1] [2 1 1 3 1 1 3 3 1 1 1 3]
Epoch 160
0.029643012
(0.29166666666666663, 0.2708333333333333)
[1 5 1 3 1 1 1 1 3 1 1 1

NameError: name 'train_process_filename' is not defined